In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import json
import serial
import numpy as np
import time

ModuleNotFoundError: No module named 'serial'

In [4]:
#File path for the dataset 
file_path ='./archive/dataset.csv'
data = pd.read_csv(file_path)

#create  label encoder
label_encoder = LabelEncoder()

#Apply label encoder to the "crop" column
data['Crop'] = label_encoder.fit_transform(data['Crop'])

print(data.head())
print(data.info())


   Crop  CropDays  SoilMoisture  temperature  Humidity  Irrigation
0     8        10           400           30        15           0
1     8         7           200           30        32           0
2     8         9           300           21        28           0
3     8         3           500           40        22           0
4     8         2           700           23        34           0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 501 entries, 0 to 500
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   Crop          501 non-null    int64
 1   CropDays      501 non-null    int64
 2   SoilMoisture  501 non-null    int64
 3   temperature   501 non-null    int64
 4   Humidity      501 non-null    int64
 5   Irrigation    501 non-null    int64
dtypes: int64(6)
memory usage: 23.6 KB
None


In [6]:
#Features and labels
X = data.drop('Irrigation', axis = 1)
y= data[['Irrigation']]

#Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

#Ensure that y_train and y_test are 1D arrays
y_train = y_train.values.ravel()
y_test = y_test.values.ravel()

In [7]:
# Train the model
model = RandomForestClassifier(n_estimators = 100, random_state = 42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [8]:
#Evaluate the model
y_pred = model.predict(X_test)
print(f'Accuracy:{accuracy_score(y_test, y_pred)}')


Accuracy:0.9207920792079208


In [9]:
#Save the model
joblib.dump(model, 'random_forest_model.pkl')

#Convert the label encoding mapping to the native Python types
crop_mapping = {str(key): int(value) for key, value in zip(label_encoder.classes_, label_encoder.transform(label_encoder.classes_))}

#Save the label encoding mapping
with open('crop_mapping.json', 'w') as f:
    json.dump(crop_mapping, f)

In [5]:
#install sklearn_porter  to convert model to c language
!pip install sklearn-porter


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.4/144.4 KB 866.9 kB/s eta 0:00:0000:0100:01


In [2]:
pip install m2cgen


Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 KB 74.1 kB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [22]:
import m2cgen as m2c

# Load your trained model
model = joblib.load('random_forest_model.pkl')

# Generate C code for the model
model_code = m2c.export_to_c(model)

# Save the model code to a file
with open('model.c', 'w') as f:
    f.write(model_code)
